In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
df=pd.read_csv('../input/online-retail-ii-uci/online_retail_II.csv', encoding = "ISO-8859-1")

In [ ]:
df

In [ ]:
afterDropNa=df.dropna(axis=0)

In [ ]:
afterDropNa.isnull().sum()

In [ ]:
afterDropNa.plot(kind='box',figsize=(20,20))

In [ ]:
df.dropna(axis=0,inplace=True)
df.drop(columns=['Invoice','StockCode','Description','Country'],inplace=True)
df['Monetary']=df.Quantity*df.Price
gb=df.groupby(['Customer ID'])
from datetime import date
recency=gb['InvoiceDate'].apply(lambda x:(pd.to_datetime(date.today())-pd.to_datetime(x.max())).days)
frequency=gb['InvoiceDate'].apply(lambda x:len(x.unique()))
monetary=gb['Monetary'].sum()
df=pd.concat([recency,frequency,monetary],axis=1,keys=['RecentTransactionDoneBefore','FrequencyOfPurchasing','Amount'])

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
ss=StandardScaler()
X=ss.fit_transform(df)
def getBestNumberOfClusters():
    from sklearn.metrics import silhouette_score
    Inertia = []
    Silhouette=[]
    kRange=np.arange(2,10)
    for K in kRange:
        model = KMeans(K)
        model.fit(X)
        Silhouette.append(silhouette_score(X,model.labels_))
        Inertia.append(model.inertia_)
    z=pd.DataFrame({"Inertia":Inertia,"Silhouette Score":Silhouette},index=kRange)
    plt.subplot(1, 2, 1)
    z.Inertia.plot(kind='line',figsize=(15,3), color='red',ylabel="Inertia",xlabel='Clusters')
    plt.subplot(1, 2, 2)
    z['Silhouette Score'].plot(kind='line',ylabel='Silhouette Score',xlabel='Clusters')
getBestNumberOfClusters()

In [ ]:
km=KMeans(3)
km.fit(X)
df['Labels']=km.labels_
df.groupby(['Labels']).agg({'Amount': ['mean', 'count'], 'FrequencyOfPurchasing': 'mean','RecentTransactionDoneBefore': 'mean'})